
# Petfinder.my - Pawpularity Contest: Simple EDA and fastai starter

In this competition, we will use machine learning to predict the "pawpularity" of a pet using images and metadata. If successful, solutions will be adapted into AI tools that will guide shelters and rescuers around the world to improve the appeal of their pet profiles, automatically enhancing photo quality and recommending composition improvements. As a result, stray dogs and cats can find families much faster, and these tools will help improve animal welfare.

In this notebook, I will present a quick 'n dirty EDA and a (image-only, for now) fastai starter. 

**As of 10/26, it's currently the best-scoring notebook for the competition, beating 10-fold ensemble models that are bigger while only using a single and smaller model.**

V0: Change get_data(fold) to correct K-Fold, use is_valid for validation data
V0: change K-Fold to 11

V1: use cleaned dataset 0fold CV:16.225714855

V3: dropped images has 100 Pawpularity

V4: add BCEWithLogitsLossFlatWeight to set different weight for different class

V5: test different weights


Here is the rmse in different bins for fold 0 for our best model

Fold 0 | Score: 17.45813597262292\
bin 0 cv: 37.49209602111998\
bin 1 cv: 22.772208169008607\
bin 2 cv: 13.207977427382245\
bin 3 cv: 10.522804120560147\
bin 4 cv: 8.483168163797071\
bin 5 cv: 9.485671151318567\
bin 6 cv: 11.804965067115697\
bin 7 cv: 15.564434605405175\
bin 8 cv: 18.247221142730872\
bin 9 cv: 23.35765871546634\
bin 10 cv: 31.41751025239491\
bin 11 cv: 35.20542376392687\
bin 12 cv: 39.36972325654612\
bin 13 cv: 43.24318517984522

Here is the rmse in different bin for fold 0 if I apply weight as norm_score * 100

Fold 0 | Score: 18.592372146023237\
bin 0 cv: 44.05764906073716\
bin 1 cv: 28.184224977673757\
bin 2 cv: 17.87528334312377\
bin 3 cv: 15.530264285947883\
bin 4 cv: 12.524930627037255\
bin 5 cv: 12.766057617128045\
bin 6 cv: 12.442131854766735\
bin 7 cv: 12.350161312759775\
bin 8 cv: 14.791775884854397\
bin 9 cv: 17.55045425213173\
bin 10 cv: 25.36814697660495\
bin 11 cv: 27.56101416173055\
bin 12 cv: 30.351763733535304\
bin 13 cv: 33.257458912599766\

If I set weight only for bin 13 like:
        weight = torch.where(weight >= 93, 30, 1)
The result is

Fold 0 | Score: 23.954549314305204\
bin 0 cv: 53.36884523690475\
bin 1 cv: 35.6026245439023\
bin 2 cv: 23.641181526441343\
bin 3 cv: 22.686988182680597\
bin 4 cv: 19.547315548671225\
bin 5 cv: 21.731360981274825\
bin 6 cv: 20.46953579887746\
bin 7 cv: 17.563321793279\
bin 8 cv: 17.185965318359507\
bin 9 cv: 16.80244003243514\
bin 10 cv: 24.518560469789914\
bin 11 cv: 22.199209214749857\
bin 12 cv: 24.355108733215193\
bin 13 cv: 24.449208502609913

If I set weight = torch.where(weight >= 93, 512, 1), I got very good rmsp for bin 11-13 as following:

Fold 0 | Score: 48.74278342925127\
bin 0 cv: 80.58898903204448\
bin 1 cv: 66.79452060481921\
bin 2 cv: 57.446016762789775\
bin 3 cv: 52.970257956221815\
bin 4 cv: 48.60457566314422\
bin 5 cv: 46.099067746755104\
bin 6 cv: 40.55932192012693\
bin 7 cv: 35.27638789027249\
bin 8 cv: 30.567799892895714\
bin 9 cv: 25.14620826375468\
bin 10 cv: 18.789601181269767\
bin 11 cv: 12.719151272973464\
bin 12 cv: 6.599604111712022\
bin 13 cv: 7.356627632656249


In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
  from google.colab import drive
  drive.mount("/gdrive", force_remount=True)

In [ ]:
if IN_COLAB:
  # !pip install -Uqq fastcore --upgrade
  !pip install --upgrade fastai==2.5.2

In [ ]:
!pip show fastai

In [ ]:
if IN_COLAB:
  !pip uninstall -y kaggle
  !pip install kaggle
  !mkdir /root/.kaggle
  !cp /gdrive/MyDrive/kaggle/kaggle.json /root/.kaggle
  !kaggle competitions download -c petfinder-pawpularity-score
  !mkdir -p ../input/petfinder-pawpularity-score
  !unzip -n -qq 'petfinder-pawpularity-score.zip' -d ../input/petfinder-pawpularity-score

  !kaggle datasets download -d  tanlikesmath/swin-transformer
  !mkdir -p ../input/swin-transformer
  !unzip -n -qq swin-transformer.zip -d ../input/swin-transformer

  !kaggle datasets download -d  kozodoi/timm-pytorch-image-models
  !mkdir -p ../input/timm-pytorch-image-models
  !unzip -n -qq timm-pytorch-image-models.zip -d ../input/timm-pytorch-image-models

#   !kaggle datasets download -d  bobber/petfinder-fastai-kf-11-weight-no-loss-v1-dec10
#   !mkdir -p ../input/petfinder-fastai-kf-11-weight-no-loss-v1-dec10
#   !unzip -n -qq petfinder-fastai-kf-11-weight-no-loss-v1-dec10.zip -d ../input/petfinder-fastai-kf-11-weight-no-loss-v1-dec10

  !kaggle datasets download -d  bobber/petfinderfastaikf11noweightnormalization
  !mkdir -p ../input/petfinderfastaikf11noweightnormalization
  !unzip -n -qq petfinderfastaikf11noweightnormalization.zip -d ../input/petfinderfastaikf11noweightnormalization

  !mkdir -p /gdrive/MyDrive/kaggle/petfinder/models
  !ln -s /gdrive/MyDrive/kaggle/petfinder/models models

## A look at the data
Let's start out by setting up our environment by importing the required modules and setting a random seed:

In [ ]:
import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
#from timm.data.mixup import Mixup

In [ ]:
from fastai.vision.all import *
#from fastai.callback.hook import *

In [ ]:
#set_seed(999, reproducible=True)
seed=999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

BATCH_SIZE = 32
NEED_TRAIN = True

N_TTA = 10

Let's check what data is available to us:

In [ ]:
dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()

We can see that we have our train csv file with the train image names, metadata and labels, the test csv file with test image names and metadata, the sample submission csv with the test image names, and the train and test image folders.

Let's check the train csv file:

In [ ]:
train_df = pd.read_csv(dataset_path/'train.csv')
train_df.head()

The metadata provided includes information about key visual quality and composition parameters of the photos. The Pawpularity Score is derived from the profile's page view statistics. This is the target we are aiming to predict.

Let's do some quick processing of the image filenames to make it easier to access:

In [ ]:
# #removed duplicates that has lower score than the same image
# removed_id = ['5ef7ba98fc97917aec56ded5d5c2b099',
#              '1feb99c2a4cac3f3c4f8a4510421d6f5',
#              '5a642ecc14e9c57a05b8e010414011f2',
#              '0422cd506773b78a6f19416c98952407',
#              '9b3267c1652691240d78b7b3d072baf3',
#              '1059231cf2948216fcc2ac6afb4f8db8',
#              '8ffde3ae7ab3726cff7ca28697687a42',
#              '78a02b3cb6ed38b2772215c0c0a7f78e',
#              'bf8501acaeeedc2a421bac3d9af58bb7',
#              'fe47539e989df047507eaa60a16bc3fd',
#              'dd042410dc7f02e648162d7764b50900',
#              '988b31dd48a1bc867dbc9e14d21b05f6',
#              'e359704524fa26d6a3dcd8bfeeaedd2e',
#              '6ae42b731c00756ddd291fa615c822a1',
#              '9a0238499efb15551f06ad583a6fa951',
#              'a9513f7f0c93e179b87c01be847b3e4c',
#              '38426ba3cbf5484555f2b5e9504a6b03',
#              'cd909abf8f425d7e646eebe4d3bf4769',
#              '9f5a457ce7e22eecd0992f4ea17b6107',
#              '3877f2981e502fe1812af38d4f511fd2',
#              'b190f25b33bd52a8aae8fd81bd069888',
#              '94c823294d542af6e660423f0348bf31',
#              '2b737750362ef6b31068c4a4194909ed',
#              '01430d6ae02e79774b651175edd40842',
#              '72b33c9c368d86648b756143ab19baeb',
#              'dbc47155644aeb3edd1bd39dba9b6953',
#              'b49ad3aac4296376d7520445a27726de',
#              '54563ff51aa70ea8c6a9325c15f55399',
#              '87c6a8f85af93b84594a36f8ffd5d6b8',
#              '16d8e12207ede187e65ab45d7def117b']

In [ ]:
len_df = len(train_df)
print(f"There are {len_df} images")
train_df['Pawpularity'].hist(figsize = (10, 5))
print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")

In [ ]:
# train_df_100 = train_df[train_df['Pawpularity']==100].copy()
# rain_df = train_df[train_df['Pawpularity']!=100].copy()

In [ ]:
len_df = len(train_df)
print(f"There are {len_df} images")
train_df['Pawpularity'].hist(figsize = (10, 5))
print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")

In [ ]:
train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train_df.head()

Okay, let's check how many images are available in the training dataset:

Let's check the distribution of the Pawpularity Score:

In [ ]:
print(f"There are {len(train_df['Pawpularity'].unique())} unique values of Pawpularity score")

Note that the Pawpularity score is an integer, so in addition to being a regression problem, it could also be treated as a 100-class classification problem. Alternatively, it can be treated as a binary classification problem if the Pawpularity Score is normalized between 0 and 1:

In [ ]:
train_df['norm_score'] = train_df['Pawpularity']/100
train_df['norm_score']

Let's check an example image to see what it looks like:

In [ ]:
im = Image.open(train_df['path'][1])
width, height = im.size
print(width,height)

In [ ]:
im

## Data loading
After my quick 'n dirty EDA, let's load the data into fastai as DataLoaders objects. We're using the normalized score as the label. I use some fairly basic augmentations here.

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'


In [ ]:
seed=999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
#Sturges' rule
num_bins = int(np.floor(1+np.log2(len(train_df))))
num_bins

In [ ]:
train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)
train_df['bins'].hist()

In [ ]:
#from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

train_df['fold'] = -1


N_FOLDS = 11
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')

train_df.fold.value_counts().plot.bar()

In [ ]:
train_df[train_df['fold']==0].head()

In [ ]:
train_df[train_df['fold']==0]['bins'].value_counts()

In [ ]:
train_df[train_df['fold']==1]['bins'].value_counts()

In [ ]:
train_df_group = train_df.groupby('bins')['Pawpularity'].agg(['min', 'max', 'mean', 'std', 'count'])
train_df_group

In [ ]:
def petfinder_rmse(input,target):
    # print('target shape', target.shape)
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
def get_data(fold):
#     train_df_no_val = train_df.query(f'fold != {fold}')
#     train_df_val = train_df.query(f'fold == {fold}')
    
#     train_df_bal = pd.concat([train_df_no_val,train_df_val.sample(frac=1).reset_index(drop=True)])
    train_df_f = train_df.copy()
    # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    
    # It looks if seed of RandomSplitter is set, it reduce the score. So we don't set seed here
    splitter = RandomSplitter(0.2)

    validate_idx = splitter(range(len(train_df)))[1]
    #find pawpularity 100
    idx_100 = train_df.loc[train_df['Pawpularity']==100].index

    # Add pawpularity 100 here
    # validate_idx = list(set(validate_idx).union(set(idx_100)))

    # print(validate_idx)
    print(f'validat dataset size is {len(validate_idx)} including {len(idx_100)} images with pawpularity 100')

    # Change RandomSplitter to IndexSplitter
    splitter = IndexSplitter(validate_idx)
    
    dls = DataBlock(blocks=(ImageBlock, RegressionBlock),
                get_x=ColReader('path'),
                get_y=ColReader('norm_score'),
                splitter=splitter,
                item_tfms=Resize(224), #pass in item_tfms
                batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(), Normalize.from_stats(*imagenet_stats)])
               )
    
    paw_dls = dls.dataloaders(train_df_f, 
                          bs=BATCH_SIZE,
                          num_workers=8,
                          seed=seed)
    
    return paw_dls, splitter


In [ ]:
#Valid Kfolder size
# the_data, splitter = get_data(0)
# assert (len(the_data.train) + len(the_data.valid)) == (len(train_df)//BATCH_SIZE)

In [ ]:
@delegates()
class BCEWithLogitsLossFlatWeight(BaseLoss):
    "Same as `nn.BCEWithLogitsLoss`, but flattens input and target."
    @use_kwargs_dict(keep=True, weight=None, reduction='mean', pos_weight=None)
    def __init__(self, *args, axis=-1, floatify=True, thresh=0.5, **kwargs):
        if kwargs.get('pos_weight', None) is not None and kwargs.get('flatten', None) is True:
            raise ValueError("`flatten` must be False when using `pos_weight` to avoid a RuntimeError due to shape mismatch")
        if kwargs.get('pos_weight', None) is not None: kwargs['flatten'] = False
        super().__init__(nn.BCEWithLogitsLoss, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)
        self.thresh = thresh

    def decodes(self, x):    return x>self.thresh
    def activation(self, x): return torch.sigmoid(x)

    def __call__(self, inp, targ, **kwargs):
        # print(f'input shape: {inp.shape}, target shape: {targ.shape}')
        # Set weight for all targets
        weight = targ * 100
        # Only set weight for bin 10
        weight = torch.where((weight >= 72) & (weight <= 78), 256, 1)
        self.func.register_buffer('weight', weight)
#         print(f'weight: {weight}')
        return super().__call__(inp, targ, **kwargs)

In [ ]:
def get_learner(fold_num):
    data, splitter = get_data(fold_num)
    
    model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=data.c)

    loss = BCEWithLogitsLossFlatWeight()
    

    learn = Learner(data, model, loss_func=loss, metrics=petfinder_rmse, cbs=[MixUp(0.2)]).to_fp16()
    
    return learn, splitter

In [ ]:
test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()

In [ ]:
if len(test_df) != 8:
    NEED_TRAIN = True

In [ ]:
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df = test_df.drop(columns=['Id'])
train_df['norm_score'] = train_df['Pawpularity']/100

In [ ]:
# if NEED_TRAIN:
#     get_learner(fold_num=0).lr_find(end_lr=3e-2)

In [ ]:
import gc

In [ ]:
from sklearn.metrics import mean_squared_error
def get_cv_for_bins(bins, pred_col='pred'):
  cv_score = mean_squared_error(train_df.loc[(train_df[pred_col]!=-1) & (train_df['bins']==bins), 'Pawpularity'], 
                                train_df.loc[(train_df[pred_col]!=-1) & (train_df['bins']==bins), pred_col], squared=False)
  return cv_score


def print_cv_for_all_bins(pred_col='pred', num_bins=13):
  cv_scores = []
  for bins in range(num_bins):
    cv = get_cv_for_bins(bins, pred_col)
    print(f'bin {bins} cv: {cv}')
    cv_scores.append(cv)

  cv_score = mean_squared_error(train_df.loc[(train_df[pred_col]!=-1), 'Pawpularity'], 
                                train_df.loc[(train_df[pred_col]!=-1), pred_col], squared=False)
  print(f'Total cv {pred_col}:{cv_score}')
  return cv_scores

In [ ]:
#####################################################################################################################
# no weight
#####################################################################################################################

In [ ]:
@delegates()
class BCEWithLogitsLossFlatWeight(BaseLoss):
    "Same as `nn.BCEWithLogitsLoss`, but flattens input and target."
    @use_kwargs_dict(keep=True, weight=None, reduction='mean', pos_weight=None)
    def __init__(self, *args, axis=-1, floatify=True, thresh=0.5, **kwargs):
        if kwargs.get('pos_weight', None) is not None and kwargs.get('flatten', None) is True:
            raise ValueError("`flatten` must be False when using `pos_weight` to avoid a RuntimeError due to shape mismatch")
        if kwargs.get('pos_weight', None) is not None: kwargs['flatten'] = False
        super().__init__(nn.BCEWithLogitsLoss, *args, axis=axis, floatify=floatify, is_2d=False, **kwargs)
        self.thresh = thresh

    def decodes(self, x):    return x>self.thresh
    def activation(self, x): return torch.sigmoid(x)

    def __call__(self, inp, targ, **kwargs):
        # print(f'input shape: {inp.shape}, target shape: {targ.shape}')
        # Set weight for all targets
#         weight = targ * 100
#         # Only set weight for bin 13
#         weight = torch.where(weight >= 93, 512, 1)
        #self.func.register_buffer('weight', weight)
        # print(f'weight: {weight}')
        return super().__call__(inp, targ, **kwargs)

In [ ]:
N_TTA = 10

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
#SVR

def forward_without_head(self, x):
    x = self.forward_features(x)
    # x = self.head(x)
    return x

def forward_org(self, x):
    x = self.forward_features(x)
    x = self.head(x)
    return x

def get_preds_for_svr(learn, df, dls=None):
  #Change forward method to get feature output
  learn.model.forward_org = learn.model.forward
  learn.model.forward = types.MethodType(forward_without_head, learn.model)

  paw_dls = dls
  if paw_dls==None:
      paw_dls = learn.dls

  # test_dl = paw_dls.test_dl(test_df)
  test_dl = paw_dls.test_dl(df)
  # test_dl = paw_dls.test_dl(train_df.loc[val_idx])
  preds, _ = learn.tta(dl=test_dl, n=N_TTA, beta=0)

  #Restore forward method
  # learn.model.forward = learn.model.forward_org
  learn.model.forward = types.MethodType(forward_org, learn.model)

  return preds

def fit_SVR(preds, target, i):
  clf = SVR(C=20.0)
  clf.fit(preds, target)

  name = f"SVR_fold_{i}.pkl"
  pickle.dump(clf, open(name, "wb"))
  print(f'Fit SVR {name}')
  return clf

def predict_SVR(model_path, preds, i, clf=None):
  if clf==None:
    name = f"{model_path}/SVR_fold_{i}.pkl"
    clf = pickle.load(open(name, "rb"))
    print(f'Load SVR {name}')
  preds_svr = clf.predict(preds)
  #reshape to right size
  preds_svr = preds_svr.reshape((-1,1))
  return preds_svr, clf

In [ ]:
META_COL = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group',
            'Collage', 'Human', 'Occlusion', 'Info', 'Blur']


In [ ]:
model_base_folder = '../input/petfinderfastaikf11noweightnormalization'

N_TTA = 10

if NEED_TRAIN:
    all_preds = []
    train_df['pred'] = -1
    train_df['preds_svr'] = -1
    train_df['preds_dual'] = -1

    for i in range(N_FOLDS):

        print(f'Fold {i} results')

#         learn, splitter = get_learner(fold_num=i)

#         learn.fit_one_cycle(5, 2e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=2)]) 
#         learn.recorder.plot_loss()  
#         learn.export(f'model_fold_{i}.pkl')
        path = f'{model_base_folder}/model_fold_{i}.pkl'
        learn = load_learner(path, cpu=False)
        print('Loaded model from ', path)

        paw_dls, splitter = get_data(i)
        # paw_dls = learn.dls
        
        test_dl = paw_dls.test_dl(test_df)

        preds, _ = learn.tta(dl=test_dl, n=N_TTA, beta=0)

        all_preds.append(preds)

        #SVR model
        preds = get_preds_for_svr(learn, test_df, paw_dls)
        #Add meta data
        preds = np.concatenate((preds.numpy(), test_df[META_COL].to_numpy()), axis=1)
        preds_svr, clf = predict_SVR('../input/petfinderfastaikf11singlepretrainsvrmeta', preds, i)
        #Norm SVR prediction
        preds_svr = preds_svr/100
        print(preds_svr[:5])
        
        all_preds.append(preds_svr)
        print('test_df.shape:', test_df.shape, 'preds_svr.shape:', preds_svr.shape)
      
        
        #Only do CV for public train
        if len(test_df) == 8:
#             val_idx = splitter(range(len(train_df)))[1]
            val_idx = np.load(f'{model_base_folder}/val_idx_{i}.npy')
            # Save validation idx here
#             np.save(f'val_idx_{i}', val_idx)
            val_df = train_df.loc[val_idx]
            val_pred, _ = learn.tta(dl=paw_dls.test_dl(val_df), n=N_TTA, beta=0)
            print(val_df['Pawpularity'][:5], val_pred[:5])
            score = mean_squared_error(val_df['Pawpularity'], val_pred*100, squared=False)
            print(f'Fold {i} | Score: {score}')
            # Save prediction of validation as pred
            train_df.loc[val_idx, 'pred'] = val_pred*100
            
            print_cv_for_all_bins()

            #SVR
            preds = get_preds_for_svr(learn, train_df.loc[val_idx], paw_dls)
            #Add meta data
            preds = np.concatenate((preds.numpy(), train_df.loc[val_idx, META_COL].to_numpy()), axis=1)
            preds_svr, clf = predict_SVR('../input/petfinderfastaikf11singlepretrainsvrmeta', preds, i, clf)
            print(val_df['Pawpularity'][:5], preds_svr[:5])
            train_df.loc[val_idx, 'preds_svr'] = preds_svr
            print_cv_for_all_bins('preds_svr')
            
            train_df.loc[val_idx, 'preds_dual'] = (train_df.loc[val_idx, 'preds_svr']+train_df.loc[val_idx, 'pred'])/2
            print(val_df['Pawpularity'][:5], val_df['preds_dual'][:5])
            print_cv_for_all_bins('preds_dual')

            
            del val_pred
        

        del learn

        torch.cuda.empty_cache()

        gc.collect()
        
        #Only run one fold for public train as we don't have so many GPU time
#         if len(test_df) == 8 and (not IN_COLAB):
#             break
    if len(test_df) == 8:
        cv_score = mean_squared_error(train_df.loc[train_df['pred']!=-1, 'Pawpularity'], 
                                      train_df.loc[train_df['pred']!=-1, 'pred'], squared=False)
        print(f'CV Score: {cv_score}')

In [ ]:
if NEED_TRAIN:
    all_preds, np.mean(np.stack(all_preds*100))
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')

if NEED_TRAIN:
    preds = np.mean(np.stack(all_preds), axis=0)
    sample_df['Pawpularity'] = preds*100
sample_df.to_csv('submission.csv',index=False)
sample_df.to_csv('submission_no_weight.csv',index=False)

In [ ]:
#####################################################################################################################
# finally
#####################################################################################################################

In [ ]:
# save CV prediction
train_df.to_csv('train_df.csv',index=False)

In [ ]:
pd.read_csv('submission.csv').head()